<a href="https://colab.research.google.com/github/johnyamarun/run-monitor/blob/main/RaceHUD_v1_0_Release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

セル 1: 環境構築とライブラリ
Googleドライブの接続と、データ解析に必要なツールを準備します。

In [ ]:
# @title 1. Setup & Drive Mount (System Fonts)
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# ライブラリ準備
!pip install -q fitparse pytz tqdm numpy

# フォントインストール（システム領域へ）
print("Processing font installation...")
!apt-get update -qq
!apt-get install -y fonts-roboto fonts-noto-cjk

# フォントパス確認
import glob
roboto_path = "/usr/share/fonts/truetype/roboto/hinted/Roboto-Bold.ttf"
noto_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Bold.ttc"

if os.path.exists(roboto_path) and os.path.exists(noto_path):
    print(f"✅ Fonts installed successfully.")
else:
    print("⚠️ Standard path not found, searching...")
    # フォールバック
    r_list = glob.glob("/usr/share/fonts/**/Roboto-Bold.ttf", recursive=True)
    n_list = glob.glob("/usr/share/fonts/**/NotoSansCJK-Bold.ttc", recursive=True)
    if r_list and n_list:
        print(f"✅ Found fonts at alternative paths.")
    else:
        print("❌ Font installation warning.")

print("\n✨ Setup Ready.")

セル 2: 究極のマスター・エンジン
このセルがメインの処理です。一番上のフォームで設定を行い、実行してください。
（iPhoneで操作する場合は、セルのメニューから**「フォーム」→「コードを非表示」**にすると非常に使いやすくなります）

In [ ]:
# @title 2. Dashboard & Mastering (Fixed Map Position) { display-mode: "form" }
# @markdown ---
# @markdown ### 📱 投稿プロファイル
PROFILE = "YOUTUBE_16_9" # @param ["YOUTUBE_16_9", "INSTAGRAM_9_16"]
HUD_POSITION = "BOTTOM" # @param ["BOTTOM", "TOP"]

# @markdown ---
# @markdown ### ⏱️ 表示モード
TIME_MODE = "LIVE (Video)" # @param ["LIVE (Video)", "TOTAL (Activity)"]

# @markdown ---
# @markdown ### 📉 表示オプション
SHOW_MAP = False # @param {type:"boolean"}
SHOW_PACE_GRAPH = False # @param {type:"boolean"}
SHOW_HR_GRAPH = False # @param {type:"boolean"}
SHOW_ELEVATION = False # @param {type:"boolean"}

# @markdown ---
# @markdown ### ⚙️ 動作設定
OVERWRITE = True # @param {type:"boolean"}
MANUAL_OFFSET_SEC = -18 # @param {type:"number"}

import glob, os, subprocess, json, bisect, shutil, math, select
import numpy as np
from datetime import datetime, timedelta, timezone
from PIL import Image, ImageDraw, ImageFont
from tqdm.notebook import tqdm
import fitparse, pytz
import xml.etree.ElementTree as ET

# ---------------------------------------------------------
# 🎨 DESIGN TOKENS
# ---------------------------------------------------------
JST = pytz.timezone("Asia/Tokyo")

class Tokens:
    C_VALUE      = (255, 255, 255, 255)
    C_VALUE_SUB  = (255, 255, 255, 200)
    C_LABEL      = (255, 255, 255, 220)
    C_PANEL      = (0, 0, 0, 105)
    C_PILL       = (0, 0, 0, 95)
    C_WIDGET_BG  = (0, 0, 0, 90)
    C_STROKE     = (255, 255, 255, 45)
    C_GRID       = (255, 255, 255, 25)
    C_LINE       = (255, 255, 255, 220)
    C_AREA       = (255, 255, 255, 40)
    RADIUS_PANEL  = 24
    RADIUS_WIDGET = 16
    RADIUS_PILL   = 10
    FONT_BOLD = "/usr/share/fonts/truetype/roboto/hinted/Roboto-Bold.ttf"
    FONT_REG  = "/usr/share/fonts/opentype/noto/NotoSansCJK-Bold.ttc"

def get_font(size, bold=True):
    path = Tokens.FONT_BOLD if bold else Tokens.FONT_REG
    if not os.path.exists(path):
        import glob
        if bold: found = glob.glob("/usr/share/fonts/**/Roboto-Bold.ttf", recursive=True)
        else: found = glob.glob("/usr/share/fonts/**/NotoSansCJK-Bold.ttc", recursive=True)
        if found: path = found[0]
    try: return ImageFont.truetype(path, int(size))
    except: return ImageFont.load_default()

# ---------------------------------------------------------
# 📐 LAYOUT ENGINE (Modified)
# ---------------------------------------------------------
def get_layout(profile, hud_pos, W, H, widget_count):
    is_yt = (profile == "YOUTUBE_16_9")
    L = {}

    # Main bar
    L["bar_w"] = int(W * (0.88 if is_yt else 0.92))
    L["bar_h"] = 110 if is_yt else 140
    L["bar_x"] = (W - L["bar_w"]) // 2

    if hud_pos == "BOTTOM":
        L["bar_y"] = H - L["bar_h"] - (40 if is_yt else 70)
    else:
        L["bar_y"] = 50 if is_yt else 200

    # Widgets
    L["gap"] = 12
    L["wid_h"] = 82 if is_yt else 78
    L["wid_w"] = L["bar_w"]
    L["wid_x"] = L["bar_x"]

    if hud_pos == "BOTTOM":
        L["stack_base_y"] = L["bar_y"] - 14
        L["stack_dir"] = -1
    else:
        L["stack_base_y"] = L["bar_y"] + L["bar_h"] + 14
        L["stack_dir"] = 1

    # Map (Layout Fix applied here)
    L["map_s"] = 300 if is_yt else 260

    if is_yt:
        # ★横型(16:9): 右側に配置しつつ、メインバーとは逆の上下位置へ逃がす
        L["map_x"] = W - L["map_s"] - 100 # 右端マージン

        if hud_pos == "BOTTOM":
            # バーが下なら、マップは右上へ
            L["map_y"] = 100
        else:
            # バーが上なら、マップは右下へ
            L["map_y"] = H - L["map_s"] - 100
    else:
        # 縦型(9:16): 従来通り
        L["map_x"] = W - L["map_s"] - 40
        total_stack_h = (L["wid_h"] + L["gap"]) * widget_count
        if hud_pos == "BOTTOM":
            L["map_y"] = max(L["stack_base_y"] - total_stack_h - L["map_s"] - 20, 220)
        else:
            L["map_y"] = L["stack_base_y"] + total_stack_h + 20

    return L

# ---------------------------------------------------------
# 🛠️ CORE UTILS
# ---------------------------------------------------------
def parse_time(s):
    if not s: return None
    s = str(s).strip().replace("Z", "+00:00")
    try: return datetime.fromisoformat(s).astimezone(JST)
    except:
        try: return datetime.strptime(s.split("+")[0], "%Y-%m-%dT%H:%M:%S").replace(tzinfo=timezone.utc).astimezone(JST)
        except: return None

def fmt_time(sec):
    sign = "-" if sec < 0 else ""
    sec = abs(int(sec))
    h, m, s = sec // 3600, (sec % 3600) // 60, sec % 60
    return f"{sign}{h}:{m:02d}:{s:02d}" if h > 0 else f"{sign}{m}:{s:02d}"

def read_exact(fd, n):
    data = b""
    while len(data) < n:
        r, _, _ = select.select([fd], [], [], 20.0)
        if not r: break
        chunk = os.read(fd, n - len(data))
        if not chunk: break
        data += chunk
    return data

def parse_fps(s):
    try:
        s = str(s)
        if "/" in s:
            a, b = s.split("/", 1); a = float(a); b = float(b)
            return a / b if b != 0 else 30.0
        return float(s)
    except: return 30.0

# ---------------------------------------------------------
# ✨ DRAWING
# ---------------------------------------------------------
def draw_pulse_dot(draw, x, y, t, intensity=1.0):
    cycle = 0.8; phase = (t % cycle) / cycle; ease = 1.0 - (1.0 - phase) ** 2
    for ring in range(3):
        r = 4 + int(24 * ease) + ring * 8
        alpha = int(170 * (1.0 - ease) * (1.0 - ring * 0.3) * intensity)
        if alpha > 0: draw.ellipse([x - r, y - r, x + r, y + r], outline=(255, 255, 255, alpha), width=2)
    core_alpha = int(205 + 50 * (1.0 - ease)); core_r = 4 + int(2 * (1.0 - ease))
    draw.ellipse([x - core_r, y - core_r, x + core_r, y + core_r], fill=(255, 255, 255, core_alpha))

def draw_pill_label(draw, cx, cy, text, font):
    b = draw.textbbox((0, 0), text, font=font); tw, th = b[2] - b[0], b[3] - b[1]
    pad_x, pad_y = 14, 5
    x0 = cx - tw / 2 - pad_x; y0 = cy - th / 2 - pad_y
    x1 = cx + tw / 2 + pad_x; y1 = cy + th / 2 + pad_y
    draw.rounded_rectangle([x0, y0, x1, y1], radius=Tokens.RADIUS_PILL, fill=Tokens.C_PILL, outline=Tokens.C_STROKE, width=1)
    draw.text((cx, cy), text, font=font, fill=Tokens.C_LABEL, anchor="mm")

def draw_widget_graph(draw, x, y, w, h, title, values, vmin, vmax, current_idx, total_points, ev_sec):
    draw.rounded_rectangle([x, y, x + w, y + h], radius=Tokens.RADIUS_WIDGET, fill=Tokens.C_WIDGET_BG, outline=Tokens.C_STROKE, width=1)
    t_font = get_font(int(h * 0.20), bold=False)
    title_w = draw.textlength(title, font=t_font)
    tx = x + 18 + title_w / 2 + 10; ty = y + 15
    draw_pill_label(draw, tx, ty, title, t_font)
    gx0, gy0 = x + 16, y + 32; gx1, gy1 = x + w - 16, y + h - 10; gw, gh = gx1 - gx0, gy1 - gy0
    for k in [1, 2]:
        gy = gy0 + gh * (k / 3); draw.line([gx0, gy, gx1, gy], fill=Tokens.C_GRID, width=1)
    step = max(1, total_points // max(80, int(gw))); denom = (vmax - vmin) if (vmax - vmin) != 0 else 1.0
    pts = []
    for j in range(0, total_points, step):
        px = gx0 + (j / (total_points - 1)) * gw if total_points > 1 else gx0
        py = gy1 - np.clip((values[j] - vmin) / denom, 0, 1) * gh
        pts.append((px, py))
    if len(pts) > 1:
        fill_pts = [(gx0, gy1)] + pts + [(pts[-1][0], gy1)]
        draw.polygon(fill_pts, fill=Tokens.C_AREA); draw.line(pts, fill=Tokens.C_LINE, width=2)
    if pts and total_points > 1:
        progress = current_idx / (total_points - 1)
        pt_idx = min(len(pts) - 1, int(progress * (len(pts) - 1)))
        cx = gx0 + progress * gw; cy = pts[pt_idx][1]
        draw_pulse_dot(draw, cx, cy, ev_sec)

def draw_map_widget(draw, mx, my, ms, stats, data, ev_sec):
    draw.rounded_rectangle([mx, my, mx + ms, my + ms], radius=Tokens.RADIUS_WIDGET, fill=Tokens.C_WIDGET_BG, outline=Tokens.C_STROKE, width=1)
    t_font = get_font(int(ms * 0.055), bold=False)
    title_w = draw.textlength("MAP", font=t_font)
    tx = mx + 18 + title_w / 2 + 10; ty = my + 15
    draw_pill_label(draw, tx, ty, "MAP", t_font)
    def to_pixel(lon, lat):
        nx = 0.5 + ((lon - stats.lon_c) * math.cos(math.radians(stats.lat_c))) / stats.m_diff
        ny = 0.5 - (lat - stats.lat_c) / stats.m_diff
        px = mx + 24 + np.clip(nx, 0, 1) * (ms - 48); py = my + 28 + np.clip(ny, 0, 1) * (ms - 52)
        return (px, py)
    route_points = [to_pixel(c[0], c[1]) for c in stats.coords[::max(1, len(stats.coords) // 300)]]
    if len(route_points) > 1: draw.line(route_points, fill=Tokens.C_LINE, width=2)
    if data["lat"] is not None and data["lon"] is not None:
        cx, cy = to_pixel(data["lon"], data["lat"])
        draw_pulse_dot(draw, cx, cy, ev_sec)

# ---------------------------------------------------------
# 🚀 MASTERING ENGINE
# ---------------------------------------------------------
ROOT = "/content/drive/MyDrive/Running_Video"
IN_F, OUT_F = os.path.join(ROOT, "Input"), os.path.join(ROOT, "Output")
os.makedirs(IN_F, exist_ok=True); os.makedirs(OUT_F, exist_ok=True)

v_files = sorted([f for f in glob.glob(IN_F + "/*") if f.lower().endswith((".mp4", ".mov"))])
t_files = sorted([f for f in glob.glob(IN_F + "/*") if f.lower().endswith((".fit", ".tcx"))])

if not v_files or not t_files:
    print("⚠️ Inputフォルダを確認してください。")
else:
    #if (not SHOW_MAP) and (not SHOW_PACE_GRAPH) and (not SHOW_HR_GRAPH) and (not SHOW_ELEVATION):
    #    SHOW_PACE_GRAPH = True; SHOW_HR_GRAPH = True; SHOW_MAP = True

    tp = t_files[0]
    ext = os.path.splitext(tp)[1].lower()
    local_tp = f"/content/temp_data{ext}"
    shutil.copy2(tp, local_tp)

    records = []

    try:
        if ext == ".fit":
            fit = fitparse.FitFile(local_tp, check_crc=False)
            messages = fit.get_messages("record")
            while True:
                try:
                    r = next(messages)
                    d = r.get_values()
                    if "timestamp" not in d: continue
                    records.append({
                        "time": d["timestamp"].replace(tzinfo=timezone.utc).astimezone(JST),
                        "speed": float(d.get("enhanced_speed", 0) or 0),
                        "dist": float(d.get("distance", 0) or 0),
                        "hr": int(d.get("heart_rate", 0) or 0),
                        "ele": float(d.get("altitude", 0) or 0),
                        "lat": d.get("position_lat"),
                        "lon": d.get("position_long"),
                    })
                except StopIteration: break
                except Exception: continue
        else:
            ns = {"ns": "http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2"}
            root = ET.parse(local_tp)
            for tp_el in root.findall(".//ns:Trackpoint", ns):
                tnode = tp_el.find("ns:Time", ns)
                ts = parse_time(tnode.text if tnode is not None else None)
                if not ts: continue
                lat = lon = None
                pos = tp_el.find("ns:Position", ns)
                if pos is not None:
                    lat_el = pos.find("ns:LatitudeDegrees", ns)
                    lon_el = pos.find("ns:LongitudeDegrees", ns)
                    if lat_el is not None and lat_el.text: lat = float(lat_el.text)
                    if lon_el is not None and lon_el.text: lon = float(lon_el.text)
                dist_el = tp_el.find("ns:DistanceMeters", ns); alt_el = tp_el.find("ns:AltitudeMeters", ns); hr_el = tp_el.find(".//ns:HeartRateBpm/ns:Value", ns)
                records.append({
                    "time": ts, "speed": 0.0,
                    "dist": float(dist_el.text) if (dist_el is not None and dist_el.text) else 0.0,
                    "hr": int(hr_el.text) if (hr_el is not None and hr_el.text) else 0,
                    "ele": float(alt_el.text) if (alt_el is not None and alt_el.text) else 0.0,
                    "lat": lat, "lon": lon,
                })
    except Exception as e:
        print(f"⚠️ Parser Error (non-fatal): {e}")

    if not records:
        raise RuntimeError("records が空です。FIT/TCXの中身を確認してください。")

    records.sort(key=lambda x: x["time"])

    for i in range(len(records)):
        if ext == ".fit":
            if records[i]["lat"] is not None: records[i]["lat"] *= (180 / 2**31)
            if records[i]["lon"] is not None: records[i]["lon"] *= (180 / 2**31)
        if i > 0 and records[i]["speed"] <= 0:
            dt = (records[i]["time"] - records[i-1]["time"]).total_seconds()
            if dt > 0:
                dd = (records[i]["dist"] - records[i-1]["dist"])
                records[i]["speed"] = max(0.0, dd / dt)

    class TStats:
        def __init__(self, recs):
            self.times = [r["time"] for r in recs]
            self.eles  = [r["ele"] for r in recs]
            self.hrs   = [r["hr"] for r in recs]
            self.paces = [min(1000/(r["speed"]*60 + 0.001), 25.0) if r["speed"] > 0.1 else 25.0 for r in recs]
            self.coords = [(r["lon"], r["lat"]) for r in recs if (r["lat"] is not None and r["lon"] is not None)]
            self.e_min, self.e_max = min(self.eles), max(self.eles)
            self.h_min, self.h_max = min(self.hrs), max(self.hrs)
            if self.coords:
                lons = [c[0] for c in self.coords]; lats = [c[1] for c in self.coords]
                self.lat_c = (max(lats) + min(lats)) / 2
                self.lon_c = (max(lons) + min(lons)) / 2
                self.m_diff = max((max(lons) - min(lons)) * math.cos(math.radians(self.lat_c)), (max(lats) - min(lats)), 0.00001)
            else:
                self.lat_c = self.lon_c = 0.0; self.m_diff = 1.0

    stats = TStats(records)
    activity_start_time = records[0]["time"]
    activity_total_sec = (records[-1]["time"] - records[0]["time"]).total_seconds()
    if activity_total_sec < 0: activity_total_sec = 0.0
    activity_total_dist_km = (records[-1]["dist"] / 1000.0) if records[-1]["dist"] else 0.0
    avg_pace_min_per_km = (activity_total_sec / 60.0) / activity_total_dist_km if activity_total_dist_km > 0 else 0.0

    def fmt_pace_from_min_per_km(p):
        if not p or p <= 0 or p > 30: return "-'--"
        m = int(p); s = int(round((p - m) * 60))
        if s >= 60: m += 1; s -= 60
        return f"{m}'{s:02d}"
    avg_pace_str = fmt_pace_from_min_per_km(avg_pace_min_per_km)

    widgets = []
    if SHOW_PACE_GRAPH: widgets.append(("PACE", stats.paces, 6.5, 2.5))
    if SHOW_HR_GRAPH:   widgets.append(("HEART RATE", stats.hrs, stats.h_min, stats.h_max))
    if SHOW_ELEVATION:  widgets.append(("ELEVATION", stats.eles, stats.e_min, stats.e_max))

    for vi, vp in enumerate(v_files):
        vn = os.path.basename(vp); out_n = f"{os.path.splitext(vn)[0]}_Final.mp4"; final_p = os.path.join(OUT_F, out_n)
        if os.path.exists(final_p) and not OVERWRITE:
            print(f"⏭️ Skip (exists): {out_n}"); continue

        print(f"\n[{vi+1}/{len(v_files)}] 🎬 Rendering: {vn}")
        res = subprocess.run(["ffprobe", "-v", "error", "-select_streams", "v:0", "-show_entries", "stream=avg_frame_rate,duration:format_tags:stream_tags", "-of", "json", vp], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if res.returncode != 0:
            print(f"❌ Video Error: {res.stderr}"); continue

        meta = json.loads(res.stdout); v_d = meta["streams"][0]
        tags = {**meta.get("format", {}).get("tags", {}), **v_d.get("tags", {})}
        fps = parse_fps(v_d.get("avg_frame_rate", "30/1"))
        dur = float(v_d.get("duration") or meta.get("format", {}).get("duration") or 0.0)

        c_t = None
        for k in ["com.apple.quicktime.creationdate", "creation_time"]:
            c_t = parse_time(tags.get(k))
            if c_t: break

        sync_t = records[0]["time"] + timedelta(seconds=float(MANUAL_OFFSET_SEC))
        if c_t and abs((c_t - records[0]["time"]).total_seconds()) < 43200:
            sync_t = c_t + timedelta(seconds=float(MANUAL_OFFSET_SEC))

        start_offset_sec = (sync_t - activity_start_time).total_seconds()
        temp_p = f"/content/temp_{out_n}"
        TW, TH = (1920, 1080) if PROFILE == "YOUTUBE_16_9" else (1080, 1920)
        vf_in = f"scale={TW}:{TH}:force_original_aspect_ratio=increase,crop={TW}:{TH},setsar=1,format=rgba"

        p_in = subprocess.Popen(["ffmpeg", "-y", "-hide_banner", "-loglevel", "error", "-i", vp, "-vf", vf_in, "-f", "rawvideo", "-vcodec", "rawvideo", "-"], stdout=subprocess.PIPE)
        p_out = subprocess.Popen(["ffmpeg", "-y", "-hide_banner", "-loglevel", "error", "-f", "rawvideo", "-vcodec", "rawvideo", "-s", f"{TW}x{TH}", "-pix_fmt", "rgba", "-r", f"{fps}", "-i", "-", "-ignore_unknown", "-dn", "-sn", "-i", vp, "-map", "0:v", "-map", "1:a:0?", "-c:v", "libx264", "-preset", "fast", "-crf", "18", "-pix_fmt", "yuv420p", "-metadata:s:v:0", "rotate=0", "-movflags", "+faststart", "-shortest", temp_p], stdin=subprocess.PIPE)

        bpf, f_cnt = TW * TH * 4, 0
        is_yt = (PROFILE == "YOUTUBE_16_9")
        L = get_layout(PROFILE, HUD_POSITION, TW, TH, len(widgets))
        pbar = tqdm(total=int(dur * fps), desc="Mastering", leave=False)

        try:
            while True:
                raw = read_exact(p_in.stdout.fileno(), bpf)
                if len(raw) != bpf: break
                base = Image.frombytes("RGBA", (TW, TH), raw)
                overlay = Image.new("RGBA", (TW, TH), (0, 0, 0, 0))
                od = ImageDraw.Draw(overlay, "RGBA")

                ev_sec = f_cnt / fps
                cur_gps_t = sync_t + timedelta(seconds=ev_sec)
                idx = min(bisect.bisect_left(stats.times, cur_gps_t), len(records) - 1)
                data = records[idx]

                # --- Main Bar ---
                od.rounded_rectangle([L["bar_x"], L["bar_y"], L["bar_x"] + L["bar_w"], L["bar_y"] + L["bar_h"]], radius=Tokens.RADIUS_PANEL, fill=Tokens.C_PANEL, outline=Tokens.C_STROKE, width=1)
                bar_h = L["bar_h"]
                val_size = int(bar_h * (0.42 if is_yt else 0.46))
                lbl_size = int(bar_h * (0.15 if is_yt else 0.16))
                unit_size = int(bar_h * (0.18 if is_yt else 0.19))
                f_val = get_font(val_size, True); f_lbl = get_font(lbl_size, False); f_unit = get_font(unit_size, False)
                val_y = L["bar_y"] + bar_h * 0.42; lbl_y = L["bar_y"] + bar_h * 0.80
                unit_y = val_y + (val_size * (0.06 if is_yt else 0.02))

                # Values
                live_time_sec = start_offset_sec + ev_sec
                total_time_sec = activity_total_sec
                pace_instant = (1000 / (data["speed"] * 60 + 0.001)) if data["speed"] > 0.1 else 0.0
                pace_instant_str = f"{int(pace_instant)}'{int((pace_instant % 1) * 60):02d}" if 0 < pace_instant < 30 else "-'--"

                time_str = fmt_time(live_time_sec) if TIME_MODE.startswith("LIVE") else fmt_time(total_time_sec)
                pace_str = pace_instant_str if TIME_MODE.startswith("LIVE") else avg_pace_str
                dist_km = data["dist"] / 1000.0

                # --- ★完成版：ゴール後の表示固定ロジック ---
                is_goal = (idx >= len(records) - 1) or (dist_km >= 42.195)

                if is_goal:
                    # 1. TIME: 活動の最終タイム（ゴールタイム）で固定
                    goal_sec = (stats.times[-1] - activity_start_time).total_seconds()
                    time_str = fmt_time(goal_sec)

                    # 2. PACE: 表示を "--'--" に固定
                    pace_str = "--'--"

                    # 3. HR: 表示を "--" に固定
                    hr_display = "--"

                    # 4. DIST: ゴール時の距離で固定（42.195kmなど）
                    # data["dist"] は idx が末尾で止まっているため、自動的に最終距離になります
                else:
                    hr_display = str(data["hr"]) if data["hr"] > 0 else "--"
                # ---------------------------------------

                items = [
                    ("TIME", time_str, None),
                    ("PACE", pace_str, "/km"),
                    ("DIST", f"{dist_km:.2f}", "km"),
                    ("HR", hr_display, "bpm")
                ]
                xs = [L["bar_x"] + L["bar_w"] * 0.125, L["bar_x"] + L["bar_w"] * 0.375, L["bar_x"] + L["bar_w"] * 0.625, L["bar_x"] + L["bar_w"] * 0.875]

                for x, (lbl, val, unit) in zip(xs, items):
                    od.text((x + 2, val_y + 2), val, font=f_val, fill=(0, 0, 0, 70), anchor="mm")
                    od.text((x, val_y), val, font=f_val, fill=Tokens.C_VALUE, anchor="mm")
                    if unit:
                        val_w = od.textlength(val, font=f_val)
                        od.text((x + val_w / 2 + 8, unit_y), unit, font=f_unit, fill=Tokens.C_VALUE_SUB, anchor="lm")
                    draw_pill_label(od, x, lbl_y, lbl, f_lbl)

                # --- Widgets ---
                base_y = L["stack_base_y"]
                for k, (title, v_list, vmin, vmax) in enumerate(widgets):
                    y_pos = base_y - (L["wid_h"] + L["gap"]) * (k + 1) if L["stack_dir"] < 0 else base_y + (L["wid_h"] + L["gap"]) * k
                    draw_widget_graph(od, L["wid_x"], y_pos, L["wid_w"], L["wid_h"], title, v_list, vmin, vmax, idx, len(records), ev_sec)

                # --- Map ---
                if SHOW_MAP and stats.coords:
                    draw_map_widget(od, L["map_x"], L["map_y"], L["map_s"], stats, data, ev_sec)

                out = Image.alpha_composite(base, overlay)
                p_out.stdin.write(out.tobytes())
                f_cnt += 1; pbar.update(1)
        finally:
            pbar.close();
            try: p_out.stdin.close()
            except: pass
            try: p_in.terminate()
            except: pass
            p_out.wait()

        if p_out.returncode == 0:
            shutil.copy2(temp_p, final_p); os.remove(temp_p)
            print(f"  ✅ Complete: {out_n}")

In [ ]:
import os
path = "/content/drive/MyDrive/Running_Video/Input"
if os.path.exists(path):
    print(f"✅ フォルダは見つかりました: {path}")
    files = os.listdir(path)
    print(f"📁 中にあるファイル: {files}")
else:
    print(f"❌ フォルダが見つかりません: {path}")
    # 親フォルダを確認
    parent = "/content/drive/MyDrive"
    if os.path.exists(parent):
        print(f"参考：MyDriveの中にあるもの: {os.listdir(parent)}")